In [6]:
import cv2
from ultralytics import solutions

In [7]:
def initialize_video(video_path):
    cap = cv2.VideoCapture(video_path)
    file_name = video_path.split("/")[-1].split(".")[0]
    output_path = f"./data/output/{file_name}_tracking.avi"

    return cap, output_path

In [8]:
def count_objects_in_region(video_path, model_path):
    """Count objects in a specific region within a video."""
    cap, output_path = initialize_video(video_path)
    assert cap.isOpened(), "Error reading video file"
    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH,
                 cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
    video_writer = cv2.VideoWriter(
        output_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

    region_points = [(430, 700),
                     (1600, 700),
                     (1600, 1080),
                     (430, 1080),
                     ]
    counter = solutions.ObjectCounter(
        show=False, region=region_points, model=model_path)

    while cap.isOpened():
        success, im0 = cap.read() # bởi vì sucess bằng false là đến cuối video rồi
        if not success:
            print(
                "Video frame is empty or video processing has been successfully completed.")
            break
        im0 = counter.count(im0)
        video_writer.write(im0)

    cap.release()
    video_writer.release()
    # cv2.destroyAllWindows()


count_objects_in_region("./data/video/highway.mp4", "./model/yolo11l.pt")

Ultralytics Solutions:  {'region': [(430, 700), (1600, 700), (1600, 1080), (430, 1080)], 'show_in': True, 'show_out': True, 'colormap': None, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'json_file': None, 'records': 5, 'show': False, 'model': './model/yolo11l.pt'}
WARNING  Environment does not support cv2.imshow() or PIL Image.show()
OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


0: 384x640 22 cars, 2 buss, 6 trucks, 4 traffic lights, 298.3ms
Speed: 3.0ms preprocess, 298.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 2 buss, 6 trucks, 4 traffic lights, 261.6ms
Speed: 2.0ms preprocess, 261.6ms inference, 1.

In [ ]:
import subprocess

def convert_avi_to_mp4(avi_file_path, output_name):
    """
    Chuyển đổi file video từ định dạng .avi sang .mp4.

    Parameters:
        avi_file_path (str): Đường dẫn tới file .avi đầu vào.
        output_name (str): Tên file .mp4 đầu ra (không bao gồm đuôi).

    Returns:
        bool: Trả về True nếu thành công, False nếu thất bại.
    """
    command = [
        "ffmpeg",
        "-i", avi_file_path,  # File đầu vào
        "-ac", "2",           # 2 kênh âm thanh
        "-b:v", "2000k",      # Bitrate video
        "-c:a", "aac",        # Codec âm thanh AAC
        "-c:v", "libx264",    # Codec video H.264
        "-b:a", "160k",       # Bitrate âm thanh
        "-vprofile", "high",  # Profile video
        "-bf", "0",           # Tắt bộ lọc B-frames
        "-strict", "experimental",  # Chế độ thử nghiệm
        "-f", "mp4",          # Định dạng đầu ra
        f"./data/output/{output_name}.mp4"  # File đầu ra
    ]

    try:
        result = subprocess.run(
            command, check=True, text=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print("Conversion successful:", result.stdout)
        return True
    except subprocess.CalledProcessError as e:
        print("Error during conversion:", e.stderr)
        return False
    

convert_avi_to_mp4('./data/output/highway_tracking.avi', "highway_tracking")